# Decision Tree

## Imports

In [48]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

## Loading Data

In [49]:
hep = pd.read_csv("data\\part2\\hepatitis", delimiter=" ")
hep_train = pd.read_csv("data\\part2\\hepatitis-test", delimiter=" ")
hep_test = pd.read_csv("data\\part2\\hepatitis-training", delimiter=" ")

display(hep.head(2))
display(hep_train.head(2))
display(hep_test.head(2))

,Class,AGE,FEMALE,STEROID,ANTIVIRALS,FATIGUE,MALAISE,ANOREXIA,BIGLIVER,FIRMLIVER,SPLEENPALPABLE,SPIDERS,ASCITES,VARICES,BILIRUBIN,SGOT,HISTOLOGY
0,live,False,False,True,True,False,False,False,False,False,False,False,True,True,True,False,False
1,live,True,False,False,True,False,True,True,False,False,True,True,True,True,True,False,False


,Class,AGE,FEMALE,STEROID,ANTIVIRALS,FATIGUE,MALAISE,ANOREXIA,BIGLIVER,FIRMLIVER,SPLEENPALPABLE,SPIDERS,ASCITES,VARICES,BILIRUBIN,SGOT,HISTOLOGY
0,live,True,True,False,True,False,True,True,True,True,False,False,True,False,True,False,True
1,die,False,False,False,True,False,False,False,True,True,False,False,True,False,True,False,True


,Class,AGE,FEMALE,STEROID,ANTIVIRALS,FATIGUE,MALAISE,ANOREXIA,BIGLIVER,FIRMLIVER,SPLEENPALPABLE,SPIDERS,ASCITES,VARICES,BILIRUBIN,SGOT,HISTOLOGY
0,live,False,False,False,True,False,False,False,True,False,True,True,True,True,True,True,False
1,die,False,False,False,True,False,False,True,True,True,True,True,False,True,False,False,True


## Data Exploration

Confirming the even split between train and test datasets.

In [50]:
print(len(hep), len(hep_train), len(hep_test))

137 25 112


No null or NA values in our dataset.

In [51]:
# check for missing values
display(np.where(pd.isnull(hep)))
display(np.where(pd.isna(hep)))

(array([], dtype=int64), array([], dtype=int64))

(array([], dtype=int64), array([], dtype=int64))

The data has loaded in as the correct types, although Class may need to be changed to factor.

In [52]:
print(hep.dtypes)
print(hep.shape)

Class             object
AGE                 bool
FEMALE              bool
STEROID             bool
ANTIVIRALS          bool
FATIGUE             bool
MALAISE             bool
ANOREXIA            bool
BIGLIVER            bool
FIRMLIVER           bool
SPLEENPALPABLE      bool
SPIDERS             bool
ASCITES             bool
VARICES             bool
BILIRUBIN           bool
SGOT                bool
HISTOLOGY           bool
dtype: object
(137, 17)


After further investigating class, it is clear this is a binary classification problem.

In [53]:
hep["Class"].unique()

array(['live', 'die'], dtype=object)

In [54]:
hep["Class"] = hep["Class"].astype('category')
hep["Class"]

0      live
1      live
2      live
3      live
4      live
       ... 
132    live
133    live
134    live
135    live
136    live
Name: Class, Length: 137, dtype: category
Categories (2, object): ['die', 'live']

In [55]:
hep_train_X = hep_train.drop("Class", axis = 1)
hep_train_y = hep_train["Class"]
hep_test_X = hep_test.drop("Class", axis = 1)
hep_test_y = hep_test["Class"]

## Decision Tree Algorithm

In [79]:
class DecisionTree:
    def __init__(self):
        import numpy as np
        import pandas as pd
        from scipy import stats
        
    def train(self, train_X: pd.DataFrame, train_y: pd.Series):
        self.train_y = train_y
        self.baseline = self.__most_common_class(train_y)
        self.class_name = train_y.name
        attributes = train_X.columns
        # concatenating x & y will make it easier for the recursive structure of the algorithm
        instances = pd.concat([train_y.to_frame(), train_X], axis = 1) # note y = column 0
        self.node = self.__build_tree(instances, attributes)
        return self.node
    
    def test(self, test_X: pd.DataFrame) -> pd.Series:
        """Tests the model on the test set and returns the predictions."""
        if(not self.node):
            raise Exception("DecisionTree not trained yet. Please call .train() first.")
        predicted_nodes = []
        for i in range(len(test_X)):
            predicted_nodes.append(self.__test_helper(test_X.iloc[i], self.node))
        return pd.Series(predicted_nodes)

    def __test_helper(self, test_X_row: pd.Series, node):
        """Helper function for the test function. Recursively traverses the tree to find the leaf node."""
        if(isinstance(node, LeafNode)):
            return node.class_name
        split_class = node.class_name
        split_value = test_X_row[split_class] # type: ignore
        if(split_value):
            return self.__test_helper(test_X_row, node.true_branch)
        return self.__test_helper(test_X_row, node.false_branch)
    
    def evaluate(self, pred: pd.Series, test_y: pd.Series):
        """Calculates the accuracy of the predictions."""
        return np.mean(pred == test_y)
    
    def __build_tree(self, instances: pd.DataFrame, attributes: pd.Index):
        """Builds an optimal structured decision tree with decision nodes and leaf nodes."""        
        # to determine the root node, we find the first question with the optimal value
        if(instances.empty):
            # returns leaf node that contains the name and probability of the most probable class
            return LeafNode(self.baseline, np.max(self.__frequency(self.train_y))) 
        unique_class_values = instances[instances.columns[0]].unique()
        if(len(unique_class_values) == 1):  # check for pure node
            # returns leaf node that contains the name of the unique class with prob 1
            return LeafNode(unique_class_values[0], 1)  
        if(attributes.empty):
            # returns leaf node that contains the name and probability of the majority class of the instances
            return LeafNode(self.__most_common_class(instances[instances.columns[0]]),  # type: ignore
                                    self.__frequency(instances[instances.columns[0]])) # type: ignore
        lowest_impurity = 1
        best_att = attributes[1]
        best_insts_true = instances
        best_insts_false = instances
        for attribute in attributes:
            # separate instances into two sets
            insts_true =  instances[instances[attribute]]
            insts_false = instances[instances[attribute] == False]
            # calculate purity of each set
            gini_true  = self.__gini_impurity(insts_true [self.class_name]) # type: ignore
            gini_false = self.__gini_impurity(insts_false[self.class_name]) # type: ignore
            # weighted average purity
            weighted_avg_impurity = ((len(insts_true)  / len(instances)) * gini_true + 
                                   (len(insts_false) / len(instances)) * gini_false)
            if(weighted_avg_impurity < lowest_impurity):
                best_att = attribute
                best_insts_true  = insts_true
                best_insts_false = insts_false
        left  = self.__build_tree(best_insts_true,  attributes.drop(best_att))
        right = self.__build_tree(best_insts_false, attributes.drop(best_att))
        node = DecisionNode(best_att, left, right)
        return node
    
    def __gini_impurity(self, class_label: pd.Series) -> float:
        """Calculates the gini impurity for the given labels."""
        freq = self.__frequency(class_label)
        return 1 - np.sum([f ** 2 for f in freq]) # type: ignore
    
    def __frequency(self, class_label: pd.Series) -> list[int]:
        """Returns a list of the frequencies of each label."""
        return [np.mean(value == class_label) for value in class_label.unique()] # type: ignore
    
    def __most_common_class(self, class_label: pd.Series):
        freq = self.__frequency(class_label)
        unique = class_label.unique()
        return unique[np.argmax(freq)]
    
class DecisionNode:
    """Decision nodes hold the optimal question at this level and two child nodes."""
    def __init__(self, class_name, true_branch, false_branch):
        self.class_name = class_name
        self.true_branch = true_branch
        self.false_branch = false_branch
        
    def report(self, indent: str):
        print(f"{indent}{self.class_name} = True: ")
        self.true_branch.report(indent+"\t")
        print(f"{indent}{self.class_name} = False: ")
        self.false_branch.report(indent+"\t")


class LeafNode:
    """Leaf nodes return the predicted class."""
    def __init__(self, class_name, probability):
        self.class_name = class_name
        self.probability = probability
    
    def report(self, indent: str):
        if(self.probability==0): 
            print(f"{indent}Unknown%n")
        else:
            print(f"{indent}Class {self.class_name}, prob={np.round(self.probability, 2)}")



## Applying the Algorithm

Lets use the hepatitis dataset to apply the decision tree algorithm.

In [80]:
dt_hep = DecisionTree()
root = dt_hep.train(hep_train_X, hep_train_y)
root.report("")

HISTOLOGY = True: 
	SGOT = True: 
		BILIRUBIN = True: 
			VARICES = True: 
				ASCITES = True: 
					Class die, prob=1
				ASCITES = False: 
					Class live, prob=1
			VARICES = False: 
				ASCITES = True: 
					SPIDERS = True: 
						Class live, prob=0.8
					SPIDERS = False: 
						SPLEENPALPABLE = True: 
							Class live, prob=1
						SPLEENPALPABLE = False: 
							Class die, prob=1
				ASCITES = False: 
					Class live, prob=0.8
		BILIRUBIN = False: 
			Class live, prob=1
	SGOT = False: 
		BILIRUBIN = True: 
			VARICES = True: 
				ASCITES = True: 
					Class live, prob=1
				ASCITES = False: 
					Class die, prob=1
			VARICES = False: 
				ASCITES = True: 
					SPIDERS = True: 
						Class live, prob=0.8
					SPIDERS = False: 
						SPLEENPALPABLE = True: 
							Class live, prob=1
						SPLEENPALPABLE = False: 
							FIRMLIVER = True: 
								BIGLIVER = True: 
									ANOREXIA = True: 
										Class live, prob=1
									ANOREXIA = False: 
										Class die, prob=1
					

In [81]:
pred = dt_hep.test(hep_test_X)
accuracy = dt_hep.evaluate(hep_test_y, pred)
print(accuracy)

0.7410714285714286


## New Data

At last, our decision tree is working on the hepatitis dataset. Let's try it on the golf dataset now.

In [ ]:
golf = pd.read_csv("data\part2\golf\golf", sep = " ")
golf_train = pd.read_csv("data\part2\golf\golf-training", sep = " ")
golf_test = pd.read_csv("data\part2\golf\golf-test", sep = " ")

display(golf.head(2))
display(golf_train.head(2))
display(golf_test.head(2))

,Class,Cloudy,Raining,Hot,Cold,Humid,Windy
0,PlayGolf,True,False,False,True,False,True
1,StayHome,True,True,False,True,False,True


,Class,Cloudy,Raining,Hot,Cold,Humid,Windy
0,PlayGolf,True,False,True,False,False,False
1,StayHome,False,False,True,False,True,True


,Class,Cloudy,Raining,Hot,Cold,Humid,Windy
0,StayHome,True,True,False,True,False,True
1,StayHome,False,False,False,False,True,False


In [ ]:
print(golf.shape)
print(np.where(pd.isnull(golf)))
print(np.where(pd.isna(golf)))
print(golf["Class"].unique())

(14, 7)
(array([], dtype=int64), array([], dtype=int64))
(array([], dtype=int64), array([], dtype=int64))
['PlayGolf' 'StayHome']


In [ ]:
golf_train_X = golf_train.drop("Class", axis = 1)
golf_train_y = golf_train["Class"]
golf_test_X = golf_test.drop("Class", axis = 1)
golf_test_y = golf_test["Class"]

In [ ]:
dt_golf = DecisionTree()
root = dt_golf.train(golf_train_X, golf_train_y)
root.report("")

Windy = True: 
	Humid = True: 
		Cold = True: 
			Class PlayGolf, prob=0.75
		Cold = False: 
			Hot = True: 
				Class StayHome, prob=1
			Hot = False: 
				Class PlayGolf, prob=1
	Humid = False: 
		Class StayHome, prob=1
Windy = False: 
	Class PlayGolf, prob=1


In [ ]:
pred = dt_golf.test(golf_test_X)
accuracy = dt_golf.evaluate(golf_test_y, pred)
print(accuracy)

0.5


## Performance Analysis

The performance of the decision tree is not great. It is only able to predict half of the test data correctly. This is not a good result.
If we implemented pruning, we would expect the decision tree to perform better.